In [1]:
from os.path import join
from os import getcwd

import numpy as np
import matplotlib.pyplot as plt

from struct2xas import Struct2XAS

In [5]:
root = r"C:\Users\jade2\Dropbox (Personal)\Jade_CNRS_Internship\Data\CIF_references"
template = r"C:\Users\jade2\Dropbox (Personal)\Jade_CNRS_Internship\heo-xas-xrd\fdmnes.tmpl"
# namesZnO_W = ['EntryWithCollCode157724', 'EntryWithCollCode154486'] # ICSD
# namesZnO_R = ['EntryWithCollCode190807', 'EntryWithCollCode57156'] # ICSD
# namesCuO_T = ['COD4105682', 'COD9008961']
# namesNiO_R = ['EntryWithCollCode9866']
# namesZnO_Sp = ['ICSD_CollCode73755', ] # I need to download these; there are many
# namesCo3O4 = ['COD9005892', 'COD1526734']
# namesCoO = ['COD1528838', 'COD1533087']
# ZnCo2O4 = 'COD5910136'
Co15Zn85O = 'COD1533020'

In [6]:
elem = 'Co'
outpath = 'FDMNES/CoO_Oh'
n = Co15Zn85O
path = join(root, n + '.cif')
s = Struct2XAS(path, elem)
sites = s.get_abs_sites()
print(sites)
s.set_abs_site(0) # corresponds to abs_index, the first element printed from the line above
# s.make_input_fdmnes(6, template=template, output_path=outpath, name=n, green=False)
s.visualize(radius=7)

[struct2xas] INFO : Frames: 1, Absorbing sites: 1. (Indexes for frames and abs_sites start at 0)
[[0, 'Zn:0.850, Co:0.150', [0.3333, 0.6667, 0.0], '2b', array([0.    , 1.8786, 0.    ]), 0.15, 0]]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[struct2xas] INFO : {'Co': 'red', 'Zn': 'green', 'O': 'blue'}
[struct2xas] WARNING : 3D displayed image does not consider partial occupancy


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\jade2\miniconda3\envs\xray\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Users\jade2\miniconda3\envs\xray\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Users\jade2\miniconda3\envs\xray\lib\logging\__init__.py", line 663, in format
    record.message = record.getMessage()
  File "C:\Users\jade2\miniconda3\envs\xray\lib\logging\__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\jade2\miniconda3\envs\xray\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\jade2\miniconda3\envs\xray\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\jade2\miniconda3\envs\xray\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_ins

In [ ]:
def plot_job(path, label):
    arr = np.genfromtxt(path, skip_header=4).transpose()
    plt.plot(arr[0], arr[1], label=label)

plot_job('FDMNES/5-cation_solidsolution/Cu/job_conv.txt', 'Cu(R)')
# plt.plot(arr[0], arr[1], label='Cu (R)')
# arr2 = np.genfromtxt('FDMNES/CuO_Tenorite/job_conv.txt', skip_header=4).transpose()
# plt.plot(arr2[0], arr2[1], label='Cu (T)')
# arr3 = np.genfromtxt('FDMNES/5-cation_solidsolution/Co/job_conv.txt', skip_header=4).transpose()
# plt.plot(arr3[0], arr3[1], label='Co (R)')
# arr = np.genfromtxt('FDMNES/CuO_Tenorite/directcif/job.txt', skip_header=4).transpose()
# plt.plot(arr[0], arr[1], label='using CIF directly')
# arr1 = np.genfromtxt('FDMNES/CuO_Tenorite/s2x/job_s2x.txt', skip_header=4).transpose()
# plt.plot(arr1[0], arr1[1], label='using struct2XAS pre-gen file')

arr = np.genfromtxt('FDMNES/COD4105682.txt', skip_header=4).transpose()
arr1 = np.genfromtxt('../Data/BM23_Averaging_PyMCA/Cu_Kedge/CuO_Aldritch_RT.csv', delimiter=',').transpose()
print(arr.shape)
print(arr1.shape)
plt.plot(arr[0], arr[1], label='CuO (T) FDMNES')
plt.plot(arr1[0]*1000 - 8984, arr1[1] * 6, label='CuO Aldritch (T) BM23')
plt.xlim(-30,250)
plt.legend()
plt.savefig('Output/CuO_T', dpi=150)
plt.show()

In [ ]:
# s.make_input_feff(5, template='feff_exafs.tmpl', parent_path='Output')

In [ ]:
# s.get_coord_envs()

In [ ]:
# s.visualize(radius=5)